In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import islice

# Point this towards the data. This notebook assumes the full data is present; if this is not
# the case, modify accordingly.
data_file = '/home/loppy/work/nscl/beta_ml/BetaScint2DEnergy.txt'

In [19]:
# Reades data from fname into a list of 16x16-arrays.
#   lines=None reads the entire file.
#   lines=(start, n) reads n lines starting from start.
#   lines=n for integer n is equivalent to lines=(0, n).
def read_data(fname, lines=None):
    with open(fname, newline='') as file:
        reader = csv.reader(file, delimiter=' ')
        data = []
      
        if type(lines) is tuple:
            sl = islice(reader, lines[0], lines[0]+lines[1])
        elif type(lines) is int:
            sl = islice(reader, 0, lines)
        else:
            sl = reader
            
        for line in sl:                
            grid = np.empty(len(line)-6, dtype=np.double)
            for i in range(grid.size):
                grid[i] = float(line[i])
            data.append((grid.reshape((16, 16)), [float(x) for x in line[-6:-3]]))
            
    return data

In [20]:
def has_neighbor(event, x, y):
    def test(xx, yy):
        nx = x + xx
        ny = y + yy
        if nx < event.shape[0] and ny < event.shape[1]:
            return (event[nx, ny] != 0).any()
        else:
            return False

    return test(0, 1) or test(0, -1) or test(1, 0) or test(-1, 0) or test(1, 1) or test(1, -1) or test(-1, 1) or test(-1, -1)

In [25]:
# This cells outputs those events for which non-zero energy readings are non-contiguous.

total = 0

# Read data_file in 1000 event chunks. Note that data_file has 1000000 events.
with open('bad.txt', 'w') as f:
    f.write("Index: Init, Total -- indiv\n")
    for i in range(0, 1000000, 1000):
    # Read the first 1000 events. Displays them as they're found.
    #for i in range(0,1000):
        data = read_data(data_file, lines=(i, 1000))
        for event_init in data:
            event, init = event_init
            nz_pixels = np.where(event != 0)
            if len(nz_pixels) != 0 \
               and len(nz_pixels[1]) != 1 \
               and not any(has_neighbor(event, x, y) for (x, y) in zip(*nz_pixels)):
                
                f.write("{}: {}, {} -- {}\n".format(total, init[0]*1000, event.sum(), event[event != 0][:]))
            total += 1